In [42]:
!pip install -q findspark
!pip install pyspark
!pip install py4j

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Github Data") \
       .getOrCreate()

spark

In [43]:
import pandas as pd

In [44]:
df1 = pd.read_csv('departments.csv')

In [45]:
df1

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,10,Administration,200,1700
1,20,Marketing,201,1800
2,30,Purchasing,114,1700
3,40,Human Resources,203,2400
4,50,Shipping,121,1500
5,60,IT,103,1400
6,70,Public Relations,204,2700
7,80,Sales,145,2500
8,90,Executive,100,1700
9,100,Finance,108,1700


In [46]:
! pip install openpyxl

In [47]:
df1.to_json('employees.json')

1•	Create a DataFrame from a JSON file and infer the schema.

In [48]:
emp_df = spark.read.json('employees.json')

In [49]:
emp_df.show()


+--------------------+--------------------+--------------------+--------------------+
|       DEPARTMENT_ID|     DEPARTMENT_NAME|         LOCATION_ID|          MANAGER_ID|
+--------------------+--------------------+--------------------+--------------------+
|{10, 20, 110, 120...|{Administration, ...|{1700, 1800, 1700...|{200, 201, 205,  ...|
+--------------------+--------------------+--------------------+--------------------+



2	Perform a left  join between two DataFrames.

In [50]:
emp_df=spark.read.csv('employees.csv',header=True, inferSchema=True)

In [51]:
dept_df=spark.read.csv('departments.csv',header=True, inferSchema=True)

In [52]:
emp_df.join(dept_df,on='Department_id',how="left").show()

+-------------+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+----------------+----------+-----------+
|DEPARTMENT_ID|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID| DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+----------------+----------+-----------+
|           50|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|  SH_CLERK|  2600|            - |       124|        Shipping|       121|       1500|
|           50|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-Jan-08|  SH_CLERK|  2600|            - |       124|        Shipping|       121|       1500|
|           10|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-Sep-03|   AD_ASST|  4400|            - |       101|  Administration|       200|       1700|
|         

3.	Use a window function to calculate the rank of employees based on their salaries within each department.

In [53]:

from pyspark.sql.window import Window

In [54]:
emp_df.withColumn('RANK', F.rank().over(Window.partitionBy('DEPARTMENT_ID').orderBy('SALARY'))).show(10)

+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+----+
|EMPLOYEE_ID|FIRST_NAME| LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|RANK|
+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+----+
|        200|  Jennifer|    Whalen| JWHALEN|515.123.4444|17-Sep-03| AD_ASST|  4400|            - |       101|           10|   1|
|        202|       Pat|       Fay|    PFAY|603.123.6666|17-Aug-05|  MK_REP|  6000|            - |       201|           20|   1|
|        201|   Michael| Hartstein|MHARTSTE|515.123.5555|17-Feb-04|  MK_MAN| 13000|            - |       100|           20|   2|
|        119|     Karen|Colmenares|KCOLMENA|515.127.4566|10-Aug-07|PU_CLERK|  2500|            - |       114|           30|   1|
|        118|       Guy|    Himuro| GHIMURO|515.127.4565|15-Nov-06|PU_CLERK|  2600|            - 

4•	Write a DataFrame to a Parquet file with partitioning by a specific column.

In [ ]:
emp_df.write.partitionBy('DEPARTMENT_ID').parquet('/content/sample_data/emp1.parquet')

In [94]:
read_df = spark.read.parquet('/content/sample_data/emp1.parquet')
read_df.show(5)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-Jan-08|SH_CLERK|  2600|            - |       124|           50|
|        120|   Matthew|    Weiss|  MWEISS|650.123.1234|18-Jul-04|  ST_MAN|  8000|            - |       100|           50|
|        121|      Adam|    Fripp|  AFRIPP|650.123.2234|10-Apr-05|  ST_MAN|  8200|            - |       100|           50|
|        122|     Payam| Kaufling|PKAUFLIN|650.123.3234| 1-May-03|  ST_MAN|  7900|            - |       100|           50|
+-----------+---

5•	Use the mapPartitions transformation to apply a function to each partition of the DataFrame.

In [57]:
dept_df.rdd.mapPartitions(lambda x: x).collect()


[Row(DEPARTMENT_ID=10, DEPARTMENT_NAME='Administration', MANAGER_ID='200', LOCATION_ID=1700),
 Row(DEPARTMENT_ID=20, DEPARTMENT_NAME='Marketing', MANAGER_ID='201', LOCATION_ID=1800),
 Row(DEPARTMENT_ID=30, DEPARTMENT_NAME='Purchasing', MANAGER_ID='114', LOCATION_ID=1700),
 Row(DEPARTMENT_ID=40, DEPARTMENT_NAME='Human Resources', MANAGER_ID='203', LOCATION_ID=2400),
 Row(DEPARTMENT_ID=50, DEPARTMENT_NAME='Shipping', MANAGER_ID='121', LOCATION_ID=1500),
 Row(DEPARTMENT_ID=60, DEPARTMENT_NAME='IT', MANAGER_ID='103', LOCATION_ID=1400),
 Row(DEPARTMENT_ID=70, DEPARTMENT_NAME='Public Relations', MANAGER_ID='204', LOCATION_ID=2700),
 Row(DEPARTMENT_ID=80, DEPARTMENT_NAME='Sales', MANAGER_ID='145', LOCATION_ID=2500),
 Row(DEPARTMENT_ID=90, DEPARTMENT_NAME='Executive', MANAGER_ID='100', LOCATION_ID=1700),
 Row(DEPARTMENT_ID=100, DEPARTMENT_NAME='Finance', MANAGER_ID='108', LOCATION_ID=1700),
 Row(DEPARTMENT_ID=110, DEPARTMENT_NAME='Accounting', MANAGER_ID='205', LOCATION_ID=1700),
 Row(DEPARTME

6 •	Implement a custom aggregation function using aggregateByKey.

In [60]:
average_salary_df = emp_df.groupBy("DEPARTMENT_ID").agg({"SALARY": "avg"})
average_salary_df.show()

+-------------+------------------+
|DEPARTMENT_ID|       avg(SALARY)|
+-------------+------------------+
|           20|            9500.0|
|           40|            6500.0|
|          100| 8601.333333333334|
|           10|            4400.0|
|           50|3721.7391304347825|
|           70|           10000.0|
|           90|19333.333333333332|
|           60|            5760.0|
|          110|           10154.0|
|           30|            4150.0|
+-------------+------------------+



7•	Perform a cross join between two DataFrames and filter the results based on a condition.

In [62]:
cross_join_df = emp_df.crossJoin(dept_df)
cross_join_df.show()
cross_join_df.count()

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+-------------+--------------------+----------+-----------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|DEPARTMENT_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+-------------+--------------------+----------+-----------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|SH_CLERK|  2600|            - |       124|           50|           10|      Administration|       200|       1700|
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|SH_CLERK|  2600|            - |       124|           50|           20|           Marketing|       201|       1800|
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|SH_CLERK|  2600|            -

1350

8•	Use the flatMap transformation to split each element of a DataFrame column into multiple rows.

In [76]:
rdd=spark.sparkContext.textFile("/content/departments.csv")
rdd.collect()

['DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID',
 '10,Administration,200,1700',
 '20,Marketing,201,1800',
 '30,Purchasing,114,1700',
 '40,Human Resources,203,2400',
 '50,Shipping,121,1500',
 '60,IT,103,1400',
 '70,Public Relations,204,2700',
 '80,Sales,145,2500',
 '90,Executive,100,1700',
 '100,Finance,108,1700',
 '110,Accounting,205,1700',
 '120,Treasury, - ,1700',
 '130,Corporate Tax, - ,1700',
 '140,Control And Credit, - ,1700',
 '150,Shareholder Services, - ,1700',
 '160,Benefits, - ,1700',
 '170,Manufacturing, - ,1700',
 '180,Construction, - ,1700',
 '190,Contracting, - ,1700',
 '200,Operations, - ,1700',
 '210,IT Support, - ,1700',
 '220,NOC, - ,1700',
 '230,IT Helpdesk, - ,1700',
 '240,Government Sales, - ,1700',
 '250,Retail Sales, - ,1700',
 '260,Recruiting, - ,1700',
 '270,Payroll, - ,1700']

In [72]:
emp_df.rdd.flatMap(lambda row: [(row['EMPLOYEE_ID'], value) for value in row['FIRST_NAME'].split(",")]).collect()
emp_df.show()


+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-Jan-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-Sep-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-Feb-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-Aug-05|    MK_REP|  6000|            - |       201|           20|


9•	Create a DataFrame from an RDD and specify the schema manually.

In [95]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

rdd = spark.sparkContext.parallelize([
    (1, "Simran", 22),
    (2, "Manas", 21),
    (3, "Aayush", 23)
])

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

df_from_rdd = spark.createDataFrame(rdd, schema)
df_from_rdd.show()


+---+------+---+
| id|  name|age|
+---+------+---+
|  1|Simran| 22|
|  2| Manas| 21|
|  3|Aayush| 23|
+---+------+---+



10•	Create a DataFrame from an RDD and specify the schema manually.[link text](https://)

In [96]:
rdd = spark.sparkContext.parallelize([
    (1, "Simran", 22),
    (2, "Manas", 21),
    (3, "Aayush", 23)
])

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

df_from_rdd = spark.createDataFrame(rdd, schema)
df_from_rdd.show()


+---+------+---+
| id|  name|age|
+---+------+---+
|  1|Simran| 22|
|  2| Manas| 21|
|  3|Aayush| 23|
+---+------+---+



11•	Use the reduceByKey transformation to sum values by key in a DataFrame.


In [99]:
rdd = emp_df.rdd.map(lambda row: (row['DEPARTMENT_ID'], row['SALARY']))
summed_rdd = rdd.reduceByKey(lambda a, b: a + b)
summed_rdd.collect()


[(50, 85600),
 (10, 4400),
 (20, 19000),
 (40, 6500),
 (70, 10000),
 (110, 20308),
 (90, 58000),
 (60, 28800),
 (100, 51608),
 (30, 24900)]

In [88]:
emp_df.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



12•	Perform a rolling average calculation using a window function.

In [100]:
from pyspark.sql.window import Window
windowSpec = Window.orderBy("EMPLOYEE_ID").rowsBetween(-2, 0)
emp_df.withColumn("rolling_avg", F.avg("SALARY").over(windowSpec)).show()


+-----------+-----------+----------+--------+------------+---------+----------+------+--------------+----------+-------------+------------------+
|EMPLOYEE_ID| FIRST_NAME| LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|       rolling_avg|
+-----------+-----------+----------+--------+------------+---------+----------+------+--------------+----------+-------------+------------------+
|        100|     Steven|      King|   SKING|515.123.4567|17-Jun-03|   AD_PRES| 24000|            - |        - |           90|           24000.0|
|        101|      Neena|   Kochhar|NKOCHHAR|515.123.4568|21-Sep-05|     AD_VP| 17000|            - |       100|           90|           20500.0|
|        102|        Lex|   De Haan| LDEHAAN|515.123.4569|13-Jan-01|     AD_VP| 17000|            - |       100|           90|19333.333333333332|
|        103|  Alexander|    Hunold| AHUNOLD|590.423.4567| 3-Jan-06|   IT_PROG|  9000|            - |       102|           6

13•	Use the foreachPartition action to write each partition of a DataFrame to a separate file.

In [102]:
def write_partition(partition):
  for row in partition:
    print(row)

emp_df.foreachPartition(write_partition)


14•	Implement a custom partitioner for an RDD and apply it to a DataFrame.






In [107]:
class CustomPartitioner(object):
  def __init__(self, num_partitions):
    self.num_partitions = num_partitions

  def __call__(self, key):
    return hash(key) % self.num_partitions

rdd = spark.sparkContext.parallelize([(1, "A"), (2, "B"), (3, "C"), (4, "D")])

partitioned_rdd = rdd.partitionBy(2,CustomPartitioner(2))

partitioned_df = partitioned_rdd.toDF(["id", "value"])

partitioned_df.write.option("header", "true").partitionBy("id").mode("overwrite").csv("partitioned_output")


15•	Perform a union of two DataFrames with different schemas and handle the schema mismatch.

In [109]:
data1 = [("Alice", 25), ("Bob", 30)]
schema1 = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])
df1 = spark.createDataFrame(data1, schema1)

data2 = [("Charlie", 35, "USA"), ("David", 40, "Canada")]
schema2 = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("country", StringType(), True)
])
df2 = spark.createDataFrame(data2, schema2)

df1_common = df1.select("name", "age")
df2_common = df2.select("name", "age")

unioned_df = df1_common.union(df2_common)
unioned_df.show()


+-------+---+
|   name|age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
|  David| 40|
+-------+---+



16•	Use the groupByKey transformation to group values by key and apply a custom aggregation function.

In [110]:
rdd = emp_df.rdd.map(lambda row: (row['DEPARTMENT_ID'], row['SALARY']))
grouped_rdd = rdd.groupByKey()

def custom_aggregation(values):
  return sum(values) / len(values)

result_rdd = grouped_rdd.mapValues(custom_aggregation)
result_rdd.collect()


[(50, 3721.7391304347825),
 (10, 4400.0),
 (20, 9500.0),
 (40, 6500.0),
 (70, 10000.0),
 (110, 10154.0),
 (90, 19333.333333333332),
 (60, 5760.0),
 (100, 8601.333333333334),
 (30, 4150.0)]

17•	Perform a right outer join between two DataFrames and handle null values in the result.

In [111]:
right_outer_join_df = emp_df.join(dept_df, on='Department_id', how="right_outer").fillna("Unknown")
right_outer_join_df.show()


+-------------+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+---------------+----------+-----------+
|DEPARTMENT_ID|EMPLOYEE_ID|FIRST_NAME| LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+---------------+----------+-----------+
|           10|        200|  Jennifer|    Whalen| JWHALEN|515.123.4444|17-Sep-03| AD_ASST|  4400|            - |       101| Administration|       200|       1700|
|           20|        202|       Pat|       Fay|    PFAY|603.123.6666|17-Aug-05|  MK_REP|  6000|            - |       201|      Marketing|       201|       1800|
|           20|        201|   Michael| Hartstein|MHARTSTE|515.123.5555|17-Feb-04|  MK_MAN| 13000|            - |       100|      Marketing|       201|       1800|
|           30|       

18•	Use the sampleBy transformation to perform stratified sampling on a DataFrame.




In [113]:
fractions = {"Sales": 0.2, "Marketing": 0.3, "IT": 0.5}
stratified_sample = emp_df.sampleBy("DEPARTMENT_ID", fractions, seed=42)

stratified_sample.show()


+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|EMAIL|PHONE_NUMBER|HIRE_DATE|JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+
+-----------+----------+---------+-----+------------+---------+------+------+--------------+----------+-------------+



•	Implement a custom UDF to perform complex string manipulations on a DataFrame column.

In [114]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def complex_string_manipulation(text):
  return text[::-1]

complex_udf = udf(complex_string_manipulation, StringType())

modified_df = emp_df.withColumn("reversed_name", complex_udf(emp_df["FIRST_NAME"]))
modified_df.show()


+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|reversed_name|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-Jun-07|  SH_CLERK|  2600|            - |       124|           50|       dlanoD|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-Jan-08|  SH_CLERK|  2600|            - |       124|           50|      salguoD|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-Sep-03|   AD_ASST|  4400|            - |       101|           10|     refinneJ|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-Feb-04|    MK_MAN| 13000|            - |       100|           20|      leahciM|
|        202|       Pat|   